Code used to log base 10 transform the data from each treatment group within each mitochondrial metric

In [ ]:
import math
import pandas as pd

In [ ]:
cols = ['Treatment-naïve (mouse 260) ',
       'AC Residual (mouse 278) ', 'DTX Residual (mouse 257) ',
       'CRB Residual (mouse 268) ', 'DTX+CRB Residual (mouse 270)']

In [ ]:
variables = ["area", "length", "mbi", "mci", "perimeter", "sphericity", "volume", "width"]
df_dict = {}
for variable in variables:
    df_dict[variable] = pd.read_csv(variable + "_data.csv", header = 1)[cols]

In [ ]:
log_df_dict = {}
for variable in variables:
    log_df_dict[variable] = df_dict[variable].applymap(math.log10)

Code used to generate qqplots as a test of normality for both the untransformed and log base 10 transformed data from each treatment group within each mitochondrial metric

In [ ]:
import statsmodels.api as sm

In [ ]:
for variable in variables:
    raw_figs = []
    transformed_figs = []
    for col in cols:
        raw_fig = sm.qqplot(df_dict[variable][col], line = "s")
        plt.title(variable + ", " + col)
        raw_figs += [raw_fig]
        
        transformed_fig = sm.qqplot(log_df_dict[variable][col], line = "s")
        plt.title(variable + ", " + col)
        transformed_figs += [transformed_fig]

    
    with PdfPages("qqplots/raw/raw_" + variable + '_qqplots.pdf') as pdf:
        for fig in raw_figs:
            pdf.savefig(fig, bbox_inches='tight') 
            plt.close(fig)
        
    with PdfPages("qqplots/transformed/transformed_" + variable + '_qqplots.pdf') as pdf:    
        for fig in transformed_figs:
            pdf.savefig(fig, bbox_inches='tight') 
            plt.close(fig)


Code used to generate kernel density estimate plots on the log base 10 transformed data for each treatment group within each mitochondrial metric

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import statistics

In [ ]:
order = ["Treatment-naïve", "AC Residual", "DTX+CRB Residual", "DTX Residual", "CRB Residual"]

In [ ]:
colors = {
    "Treatment-naïve": "tab:gray",
    "AC Residual": "tab:red",
    "DTX Residual": "tab:green",
    "CRB Residual": "tab:purple",
    "DTX+CRB Residual": "tab:blue"
}

In [ ]:
for variable in variables:
    df = log_df_dict[variable]
    naive_median = statistics.median(df['Treatment-naïve (mouse 260) '])
    long_df = pd.DataFrame(columns = ["treatment", "value"])
    
    treatment = []
    value = []
    
    for col in df.columns:
        value += list(df[col])
        treatment += [col[:col.index(" (")] for i in range(len(df[col]))]
        
    long_df["treatment"] = treatment
    long_df["value"] = value
    
    dimensions_dict = {"wide": {"height": 3, "aspect": 1}, "skinny": {"height": 1.7, "aspect": 4}}
    
    for size in dimensions_dict.keys():
        plot = sns.FacetGrid(long_df, row = "treatment", row_order = order, hue = "treatment", palette = colors, hue_order = order,
                         height=dimensions_dict[size]["height"], aspect=dimensions_dict[size]["height"], legend_out = True)
        plot.refline(x=naive_median)
        plot.map(sns.kdeplot, "value", fill = True)
        plot.add_legend(title = "")
        plot.set_titles(template = "")
        plot.set_axis_labels(x_var = variable[0].upper() + variable[1:], y_var = "")

        plt.savefig("final_plots/"+ variable + "_log_densityplot_stacked_" + size + ".pdf", bbox_inches = "tight")
        plt.clf()
    
  